In [ ]:
import numpy as np
n = 100
mA = [1.0, 0.5]
sigmaA = 0.5
mB = [-1.0, 0.0]
sigmaB = 0.5
classA = np.ndarray(shape=(2,n))
classB = np.ndarray(shape=(2,n))
classA[0,:] = np.random.normal(0,1,(1,n)) * sigmaA + mA[0]
classA[1,:] = np.random.normal(0,1,(1,n)) + sigmaA + mA[1]
classB[0,:] = np.random.normal(0,1,(1,n)) * sigmaB + mB[0]
classB[1,:] = np.random.normal(0,1,(1,n)) * sigmaB + mB[1]
data = np.concatenate((classA, classB), axis=1)
targets_for_perceptron = np.array([1 for i in range(n)] + [0 for i in range(n)])
targets = np.array([1 for i in range(n)] + [-1 for i in range(n)])
targets1 = targets_for_perceptron.reshape(1,-1)
data_targets = np.concatenate((data, targets1), axis=0)
np.random.shuffle(data_targets.transpose())
data = data_targets[:2,:]
targets = data_targets[2,:]

In [ ]:
import numpy as np


# patterns = input matrix (d_in x n+1) + 1 for bias term
# targets = output matrix of labels (d_out x n) (d_out = 1 in this case),
# for perceptron learning, targets should be 0,1,
# for delta rule, targets should be -1 or 1

class Single_layer:

    def __init__(self, num_data_points, input_dims, output_dims, use_bias=True, learning_rate=0.01):
        if use_bias:
            self.bias = 1
        else:
            self.bias = 0
        # self.patterns = np.ones((num_data_points+bias, input_dims))
        np.random.seed(0)
        self.weights = np.random.normal(0, 1, (1, input_dims + self.bias))
        self.learning_rate = learning_rate

    def forward(self, data):

        output = np.dot(self.weights, data)
        #print(output)

        # threshold
        # output = np.where(output < 0, -1, 1)
        return output

    def perceptron_rule(self, data, output, targets):
        # weight update is x if positive class is misclassified as negative,
        # -x if negative class is misclassified as positive
        # thresholding:
        output = np.where(output < 0, 0, 1)
        weight_update = np.sum((targets-output) * data, axis=1)
        self.weights += self.learning_rate * weight_update
        misclassified = np.sum((targets-output) ** 2) / len(targets)
        return misclassified

    def delta_rule(self, data, output, targets):
        #print(output)
        #print(targets)
        error = targets - output
        #print(error)
        weight_update = np.dot(error, np.transpose(data))
        self.weights += self.learning_rate*weight_update
        return np.sum(error ** 2) / 2

    def training_perceptron_rule(self, data, targets, epochs=20):
        if self.bias == 1:
            ones = np.ones((1, data.shape[1]))
            data = np.concatenate((data, ones), axis=0)
        # epochs = 20
        d_error = 1000
        old_error = 1000
        epoch = 1
        while epoch < epochs or d_error > 0.001:
            output = self.forward(data)
            error = self.perceptron_rule(data, output, targets)
            print("training epoch " + str(epoch), "error: ", error)
            d_error = abs(old_error - error)
            old_error = error
            epoch++
            

    def training_delta_rule(self, data, targets, epochs=20, batch_learning=True):
        if self.bias == 1:
            ones = np.ones((1, data.shape[1]))
            data = np.concatenate((data, ones), axis=0)
        if batch_learning:
            for epoch in range(epochs):
                output = self.forward(data)

                error = self.delta_rule(data, output, targets)
                print("training epoch: ", epoch, error)

        else:
            for epoch in range(epochs):
                for sample in range(data.shape[1]):
                    input = np.reshape(data[:, sample], (data.shape[0], 1))
                    output = self.forward(input)

                    error = self.delta_rule(data, output, targets[sample])


Perceptron Rule experiments:

In [ ]:
import matplotlib.pyplot as plt

single = Single_layer(200, 2, 1)
single.training_perceptron_rule(data, targets)
weights = single.weights
x1 = np.linspace(-2,2,100)
x2 = np.linspace(-2,2,100)
b = np.ones(100)
x_range = np.concatenate((x1.reshape(1,-1),x2.reshape(1,-1),b.reshape(1,-1)),axis=0)
y_range = np.dot(weights, x_range)
print(y_range)
zeros = np.ones((1,100))
#plt.scatter(data[0],data[1], c=targets)
plt.axis([-2,2,-2,2])
plt.plot(zeros, y_range, c = "red")
plt.show()


In [ ]:
single2 = Single_layer(100, 2, 1)
single2.training_delta_rule(data, targets)

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(data[0], data[1], c=targets)
plt.show()

